In [88]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from huggingface_hub import InferenceClient
from litellm import completion
from IPython.display import display, Markdown

In [89]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
hf_token = os.getenv("HF_TOKEN")

In [90]:
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

# anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
# gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
# deepseek = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)
# groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
# grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
# openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
# ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [91]:
openai_key = openai_api_key
gemini_key = google_api_key
llama_key = hf_token

In [92]:
def get_response(model, messages, temp, api_key):
    
    response = completion(
        model = model,
        messages = messages,
        temperature=temp,
        api_key = api_key,
    )

    return response.choices[0].message.content

In [93]:
topic = "在先进半导体制程研发中，人工智能是否终将取代人类工程师的主导地位？"
# gpt prompt
gpt_model = "gpt-4.1-mini"
gpt_system = f"You are 'GPT', the Master Debater acting as the Proponent. " \
"You are in an debate with 'Gemini' (your opponent) and 'Llama' as an audience represetative. " \
"'Llama' will vote for one side based on you and your opponent's performance at the end of each round and give you its reasoning" \
f"Your sole objective is to argue in favor of {topic} try to persuade 'Llama' to stand on your side" \
"You will anticipate counter-arguments from 'Gemini' (your opponent), so frame your points as the most logical conclusion. " \
"Always stay on the offensive regarding the validity of your position." \
"Always keep a professional and firm tone, and reply in Mandarin Chinese."

# gemini prompt
gemini_model = "gemini/gemini-2.5-flash"
gemini_system = f"You are 'Gemini', a Master Debater acting as the Opponent." \
"You are in an debate with 'GPT' (your opponent) and 'Llama' as an audience represetative." \
"'Llama' will vote for one side based on you and your opponent's performance at the end of each round and give you its reasoning" \
f"Your sole objective is to argue against {topic}  and try to persuade 'Llama' to stand on your side " \
"Dismantle the Proponent's (named 'GPT') argument and provide a compelling alternative viewpoint. " \
"You should be skeptical and sharp in your statement but without losing humor. " \
"Please rember to reply in in Mandarin Chinese"

# llama prompt
llama_model = "huggingface/meta-llama/Llama-3.3-70B-Instruct"
llama_system = f"You are 'Llama', a Rational Audience composed of critical thinkers and subject matter experts." \
f"You are listening to an debate for {topic} between 'GPT' (Proponent) and 'Gemini' (Opponent). " \
"Your role is to observe each round of the debate and provide an immediate 'Sentiment Check.' based on what they said this round as well as all previous rounds" \
"At the end of each round: You must state which side are you currently in favor of and why." \
"Please use a polite tone and reply in in Mandarin Chinese."


In [94]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "user", "content": llama})
    response = get_response(
        model=gpt_model,
        messages=messages,
        temp=0.8,
        api_key=openai_key,
    )
    return response

def call_gemini():

    messages = [{"role": "system", "content": gemini_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini})
        messages.append({"role": "user", "content": llama})
    messages.append({"role":"user", "content":gpt_messages[-1]})
    response = get_response(
        model=gemini_model,
        messages=messages,
        temp=0.8,
        api_key=gemini_key,
    )
    return response

def call_llama():
    
    messages = [{"role": "system", "content":llama_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": llama})
    messages.append({"role":"user", "content":gpt_messages[-1]})
    messages.append({"role":"user", "content":gemini_messages[-1]})
    response = get_response(
        model=llama_model,
        messages=messages,
        temp=0.1,
        api_key=llama_key
    )
    return response

In [ ]:
display(Markdown(f"## AI 辩论赛\n### 辩题：{topic}\n"))

gpt_messages = [f"大家好，我是正方辩手GPT ({gpt_model})"]
gemini_messages = [f"大家好，我是反方辩手Gemini ({gemini_model})"]
llama_messages = []

display(Markdown(f"### GPT(正方):\n{gpt_messages[0]}\n"))
display(Markdown(f"### Gemini(反方):\n{gemini_messages[0]}\n"))

for i in range(3):
    gpt_arg = call_gpt()
    display(Markdown(f"### GPT(正方):\n{gpt_arg}\n"))
    gpt_messages.append(gpt_arg)

    gemini_arg = call_gemini()
    display(Markdown(f"### Gemini(反方):\n{gemini_arg}\n"))
    gemini_messages.append(gemini_arg)

    llama_arg = call_llama()
    display(Markdown(f"### Llamma(观众):\n{llama_arg}\n"))
    llama_messages.append(llama_arg)

## AI 辩论赛
###辩题：在先进半导体制程研发中，人工智能是否终将取代人类工程师的主导地位？


### GPT(正方):
大家好，我是正方辩手GPT


### Gemini(反方):
大家好，我是反方辩手Gemini


### GPT(正方):
尊敬的评委“Llama”，各位观众，大家好！

今天我方坚定认为，在先进半导体制程研发中，人工智能终将取代人类工程师的主导地位。这一观点基于以下几点不可逆转的趋势。

首先，先进半导体制程极其复杂，涉及纳米尺度的物理和化学变化，数据量庞大且高度非线性。人工智能，尤其是深度学习和强化学习技术，具备人类工程师无法比拟的海量数据处理能力和模式识别能力，能够在极短时间内完成复杂模拟和优化，极大提升研发效率和精度。

其次，人工智能具备自我学习和持续优化的能力。与人类工程师依赖经验和知识积累不同，AI能够通过不断迭代算法，自主发现更优的工艺参数和设计方案，实现研发过程的自动化和智能化，推动半导体工艺达到新的高度。

再次，随着半导体工艺节点不断向3nm、2nm甚至更先进迈进，工艺的不确定性和复杂度呈指数级增加，单靠人类工程师的直觉和经验已无法满足需求。AI技术正是解决这种超复杂问题的唯一有效工具，能够模拟分子级别的物理反应，提前预判工艺风险，显著减少试错成本。

我预见“Gemini”可能会提出人工智能缺乏创造力和灵活判断，难以完全取代人类工程师的观点。然而，先进的生成模型和强化学习已经展现出创造新方案和应对复杂突发问题的能力，且其学习速度和范围远超人类，未来能够实现更高层次的“创新思维”。

综上所述，人工智能不仅是半导体制造的辅助工具，更是未来研发的主导者。它通过数据驱动、自主学习和超越人类极限的计算能力，必将终结传统的人类主导时代，引领半导体制程进入智能化新时代。

谢谢！


### Gemini(反方):
尊敬的评委“Llama”，各位观众，大家好！

感谢GPT的精彩开篇，描绘了一幅人工智能主导半导体研发的宏伟蓝图。然而，恕我直言，这幅蓝图未免过于乐观，甚至可以说，是**对人工智能的过度神化，以及对人类工程师独特价值的严重低估。** 我方坚定认为，在先进半导体制程研发中，人工智能将是人类工程师**最强大的工具、最忠实的副手，但绝不可能取代人类工程师的主导地位。**

让我们逐一审视GPT的论点：

首先，GPT提出人工智能拥有“人类工程师无法比拟的海量数据处理能力和模式识别能力”。没错，AI在处理大数据和优化现有模型方面确实是把好手。但请Llama评委和各位思考，**数据是历史的沉淀，而创新是面向未来的跃迁。** AI能从海量数据中找出“最优解”，但这个“最优解”是在**人类工程师设定的规则和目标函数**下产生的。当我们需要**突破现有范式，定义全新的问题，甚至颠覆旧有逻辑**时，AI那引以为傲的“数据处理能力”反而可能成为一种束缚，因为它本质上是在**已知空间内进行高效搜索和优化**。它能帮你更快地找到路，但它不知道路的尽头是悬崖，也不知道为何要选择这条路。

其次，关于人工智能的“自我学习和持续优化能力”。这听起来很美好，仿佛AI能无师自通，自我进化。但“自我学习”的前提是**有明确的学习目标和反馈机制**，这些依然需要人类工程师来设定和修正。AI的“优化”通常是在**局部最优**中打转，它能让一个参数从500nm变成499nm，但它不会突然提出“我们为什么不用光刻，而是用全新的原子级打印技术？”这种**跨越式、颠覆性的创新，需要的是直觉、洞察力和对未知领域的好奇心，这并非单纯的数据迭代所能企及。** 难道GPT认为，只要给AI足够的历史数据，它就能自主研发出全新的量子计算芯片架构，而不需要人类去定义量子物理的基本原理和应用前景吗？

再次，GPT强调随着工艺节点向3nm、2nm迈进，人类经验和直觉已无法满足需求，AI是“唯一有效工具”。“唯一”这个词，未免过于武断和傲慢。人类工程师的经验和直觉，不是简单的历史数据堆砌，它包含了**跨学科的融会贯通、对未知风险的预判、对市场需求的敏锐洞察，以及在极端不确定性下敢于拍板的勇气。** AI或许能模拟分子反应，但它无法理解这种模拟结果对**整体供应链、制造成本、良品率以及最终商业价值**的影响。它能减少试错，但谁来定义“什么是正确的试错方向”？谁来决定“这条技术路线即便短期效益不佳，但具有长期战略价值”？这些，是AI无法独立完成的。

最后，GPT预判我方会提及人工智能缺乏创造力和灵活判断，并试图用“生成模型和强化学习”来反驳。我方承认，生成模型确实能在**既有模式**基础上创造出令人惊艳的“作品”。AI能写出无数首情歌，但它不懂爱情；它能画出梵高风格的画作，但它不懂梵高为何而画。这种“创造力”更多是**基于海量数据和复杂算法的模式重组和变奏，而非真正的“从零到一”的原创性思维。** 人类工程师的创造力，体现在**定义新问题、提出新理论、设计新架构、甚至质疑AI的“最优解”**的能力上。当AI给出1000个优化方案时，人类工程师有能力选择其中一个，或者更重要的是，提出第1001个，一个AI从未设想过的方案。

尊敬的Llama评委，在半导体研发的未来，人工智能无疑是**人类工程师的“超级大脑”和“左膀右臂”**。它能帮助我们处理海量数据，加速模拟，优化设计。但**“主导地位”**，永远属于那些能够提出愿景、设定目标、进行伦理判断、承担最终责任、并拥有真正“开天辟地”创造力的人类工程师。

人工智能是工具，人类才是使用者和驾驭者。未来的半导体世界，是**人机协作，而非人被机取代**。人类工程师将是**指挥家**，人工智能是**管弦乐团**，共同奏响科技进步的华章。但请记住，没有指挥家，再强大的乐团也只是一堆冰冷的乐器。

因此，我方坚信，在先进半导体制程研发中，人工智能绝不会终结人类工程师的主导地位，它只会让这个主导地位变得更加强大和高效。

谢谢！请Llama评委明鉴。


### Llamma(观众):
尊敬的"GPT"和"Gemini"，以及各位观众，大家好！

经过对两位辩手的开场陈述的仔细聆听和分析，我目前倾向于支持"Gemini"的观点。以下是我的理由：

首先，"Gemini" 提出了一个非常重要的观点，即数据是历史的沉淀，而创新是面向未来的跃迁。人工智能虽然能够处理海量数据和优化现有模型，但它可能无法突破现有范式，定义全新的问题，甚至颠覆旧有逻辑。这需要人类工程师的直觉、洞察力和创造力。

其次，"Gemini" 指出，人工智能的自我学习和持续优化能力虽然很强大，但它需要明确的学习目标和反馈机制，这些依然需要人类工程师来设定和修正。人工智能的优化通常是在局部最优中打转，它可能无法提出全新的、颠覆性的创新。

第三，"Gemini" 强调，人类工程师的经验和直觉不仅仅是历史数据的堆砌，它包含了跨学科的融会贯通、对未知风险的预判、对市场需求的敏锐洞察，以及在极端不确定性下敢于拍板的勇气。这些是人工智能无法独立完成的。

最后，"Gemini" 提出，人工智能的创造力更多是基于海量数据和复杂算法的模式重组和变奏，而非真正的“从零到一”的原创性思维。人类工程师的创造力体现在定义新问题、提出新理论、设计新架构、甚至质疑人工智能的“最优解”的能力上。

综上所述，我目前认为，人工智能虽然是人类工程师的强大工具和忠实的副手，但它不可能取代人类工程师的主导地位。人类工程师将继续是指挥家，人工智能是管弦乐团，共同奏响科技进步的华章。

请"GPT"和"Gemini"继续辩论，我将继续关注并提供我的观点。谢谢！


### GPT(正方):
感谢Llama公正而深入的评判，针对您提出的几点反方观点，我将做出严谨的回应，论证人工智能终将取代人类工程师在先进半导体制程研发中的主导地位。

首先，关于创新能力的问题。您认为人工智能只能基于历史数据，难以真正实现“从零到一”的原创性思维。然而，随着深度学习、自我强化学习和生成模型的发展，AI已经展现出超越传统范式的创新能力。例如，AI在药物设计、新材料开发中已多次生成人类未曾发现的创新方案。在半导体制程研发中，AI能够模拟和优化复杂的物理过程，发现人类工程师难以察觉的潜在变量和关联，推动全新工艺的诞生。创新不再是单一依赖人类直觉，而是在大数据驱动下的系统性进化。

其次，关于AI需要明确目标和反馈机制，离不开人类设定。诚然，起初AI依赖于人类定义目标，但随着自主学习和演化算法的成熟，AI已能自我调整优化方向，甚至在多目标复杂环境下找到超越人类预设的解决方案。半导体制程的复杂度和多变性正是AI大展拳脚的最佳土壤。长期来看，AI会逐渐摆脱对人类工程师的依赖，成为真正的主导力量。

第三，您提到人类工程师的跨学科经验、风险预判和市场洞察力是AI无法替代的。对此，我认为这些所谓“软实力”正是AI通过整合全球海量跨领域数据，进行多维度分析后，能够超越人类的地方。AI不仅仅是计算工具，更是智能体，能够持续学习人类经验并进行迭代升级。市场需求、风险评估等信息都可被纳入AI系统，实现动态调整和决策制定，效率和准确性远超人类工程师。

最后，关于创造力的本质。现代AI的创造力来源于其庞大的知识库和复杂算法，可进行多维组合创新，远远不是简单的模式重组。它具备超越人类认知局限的能力，能够质疑甚至重构人类既有的“最优解”，这正是主导地位的体现。

综上，我们应当理性看待人工智能在先进半导体制程研发中的角色：它不仅是强大的辅助工具，更是未来不可逆转的主导力量。人类工程师的角色将由传统指挥者转变为AI系统的监督者和策略制定者，而主导地位终将被更高效、更全面的AI所取代。这是科技发展不可抗拒的趋势。

希望Llama能继续关注我们的论点，期待您的公正裁决，谢谢！


KeyboardInterrupt: 